## 対数線形モデルクラスの定義

In [52]:
import torch
from torch import nn
from torch import optim

class LogLinear(nn.Module):
    """
    対数線形モデルクラス
    """
    def __init__(self):
        super(LogLinear, self).__init__()
        self.linear = nn.Linear(3, 3) # wx+b 初期パラメータはランダム

        # パラメータの設定
        self.linear.weight = torch.nn.Parameter(torch.Tensor([[0.1, -0.3, -0.2], [-0.4, -0.1, 0.7], [-0.3, 0.4, 0.1]]))
        self.linear.bias = torch.nn.Parameter(torch.Tensor([0.4, 0.5, -0.1]))

        # 損失関数
        self.loss_func = nn.CrossEntropyLoss()

        
    def forward(self, input_vector):
        """
        順伝搬
        :param input_vector: 入力ベクトル
        :return wx: モデルの出力
        :return prob: 各クラスの確率値
        """
        wx = self.linear(input_vector)
        prob = nn.functional.softmax(wx, dim=1)
        return wx, prob

    
    def forward_loss(self, input_vector, label):
        """
        順伝搬 + 損失計算
        :param input_vector: 入力ベクトル
        :return loss:損失
        """
        wx, prob = self.forward(input_vector)
        loss = self.loss_func(wx, label)
        return loss

In [67]:
# モデルのインスタンス
log_linear_model = LogLinear()

## 最急勾配法による最適化

In [68]:
# 損失の最小化
op = optim.SGD(log_linear_model.parameters(), lr=1.0) # lr:learning rate (学習率)

In [69]:
# SGDの引数にはlog_linear中の学習パラメータ一覧を渡している
list(log_linear_model.parameters())

[Parameter containing:
 tensor([[ 0.1000, -0.3000, -0.2000],
         [-0.4000, -0.1000,  0.7000],
         [-0.3000,  0.4000,  0.1000]], requires_grad=True),
 Parameter containing:
 tensor([ 0.4000,  0.5000, -0.1000], requires_grad=True)]

## 順伝搬

In [70]:
# 入力とラベル
input_vector = torch.tensor([[2, 3, 4]], dtype=torch.float)
label = torch.tensor([2], dtype=torch.long)

In [71]:
# 順伝搬と損失計算
loss = log_linear_model.forward_loss(input_vector, label)

## 逆伝搬

In [72]:
# 逆伝搬 (勾配の設定)
loss.backward()

In [73]:
# 勾配の確認
print(log_linear_model.linear.weight.grad)
print(log_linear_model.linear.bias.grad)

tensor([[ 0.0563,  0.0845,  0.1127],
        [ 1.5274,  2.2911,  3.0548],
        [-1.5837, -2.3756, -3.1675]])
tensor([ 0.0282,  0.7637, -0.7919])


## パラメータ更新
$ w_{new} = w + lr * w_{grad}$

In [77]:
# 更新前パラメータの確認
print(log_linear_model.linear.weight)
print(log_linear_model.linear.bias)

Parameter containing:
tensor([[ 0.0437, -0.3845, -0.3127],
        [-1.9274, -2.3911, -2.3548],
        [ 1.2837,  2.7756,  3.2675]], requires_grad=True)
Parameter containing:
tensor([ 0.3718, -0.2637,  0.6919], requires_grad=True)


In [78]:
# gradに基づくパラメータ更新
op.step()

In [79]:
# 更新後パラメータの確認
print(log_linear_model.linear.weight)
print(log_linear_model.linear.bias)

Parameter containing:
tensor([[-0.0127, -0.4690, -0.4253],
        [-3.4548, -4.6822, -5.4096],
        [ 2.8675,  5.1512,  6.4349]], requires_grad=True)
Parameter containing:
tensor([ 0.3437, -1.0274,  1.4837], requires_grad=True)


In [80]:
# 勾配の消去
log_linear_model.zero_grad()

## 学習：勾配に基づくパラメータ更新の繰り返し
順伝搬 -> 損失計算 -> 逆伝搬 -> 勾配計算 -> パラメータ更新 -> 順伝搬 -> ・・・

In [83]:
for epoch in range(3): # 今回は3回に設定　普通100など収束のために十分な数を設定
    print("epoch: ", epoch)

    # 順伝搬と損失計算
    loss = log_linear_model.forward_loss(input_vector, label)
    print("loss",loss)

    # パラメータの確認
    print("更新前パラメータ")
    print(log_linear_model.linear.weight)
    print(log_linear_model.linear.bias)

    # 逆伝搬 (勾配の設定)
    loss.backward()

    # 勾配の確認
    print("勾配")
    print(log_linear_model.linear.weight.grad)
    print(log_linear_model.linear.bias.grad)

    # パラメータの更新
    op.step()

    # 勾配の消去
    log_linear_model.zero_grad()

    # パラメータの確認
    print("更新後パラメータ")
    print(log_linear_model.linear.weight)
    print(log_linear_model.linear.bias)
    print("="*10)


epoch:  0
loss tensor(0., grad_fn=<NllLossBackward>)
更新前パラメータ
Parameter containing:
tensor([[-0.0127, -0.4690, -0.4253],
        [-3.4548, -4.6822, -5.4096],
        [ 2.8675,  5.1512,  6.4349]], requires_grad=True)
Parameter containing:
tensor([ 0.3437, -1.0274,  1.4837], requires_grad=True)
勾配
tensor([[1.1594e-22, 1.7391e-22, 2.3188e-22],
        [2.1437e-40, 3.2155e-40, 4.2873e-40],
        [0.0000e+00, 0.0000e+00, 0.0000e+00]])
tensor([5.7970e-23, 1.0718e-40, 0.0000e+00])
更新後パラメータ
Parameter containing:
tensor([[-0.0127, -0.4690, -0.4253],
        [-3.4548, -4.6822, -5.4096],
        [ 2.8675,  5.1512,  6.4349]], requires_grad=True)
Parameter containing:
tensor([ 0.3437, -1.0274,  1.4837], requires_grad=True)
epoch:  1
loss tensor(0., grad_fn=<NllLossBackward>)
更新前パラメータ
Parameter containing:
tensor([[-0.0127, -0.4690, -0.4253],
        [-3.4548, -4.6822, -5.4096],
        [ 2.8675,  5.1512,  6.4349]], requires_grad=True)
Parameter containing:
tensor([ 0.3437, -1.0274,  1.4837], requ